In [1]:
import sys
sys.path.append('/Users/ianleefmans/Desktop/stock-sentiment/utils')
from data import Database, ScrapeWSB, Stock
%cd /Users/ianleefmans/Desktop/stock-sentiment

/Users/ianleefmans/Desktop/stock-sentiment


In [2]:
import transformers
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [37]:

db = Database()
db.use_database('DB1')

print(db.query('show tables;'))

connection established
[(datetime.datetime(2021, 5, 28, 16, 21, 30),)]
[('COMMENTS',), ('POSTS',), ('STOCKS',)]


In [17]:
print(db.query('''SELECT * FROM STOCKS ;'''))

[('GME', datetime.datetime(2021, 5, 14, 1, 43, 41))]


In [ ]:
indexes = db.query('''SELECT POST_ID FROM POSTS;''')

In [39]:
db.query("SELECT TITLE FROM POSTS WHERE POST_ID='{}' AND STOCK_ID='{}'".format(indexes[0][0], 'GME'))[0][0]

"I, too, YOLO'D my retirement account into GME when I switched jobs. PICK ME UP ON THE WAY TO THE MOO"

In [45]:
indices = db.query("SELECT POST_ID FROM POSTS WHERE StOCK_ID='{}';".format("GME"))

In [62]:
db.query("SELECT COUNT(*) FROM POSTS WHERE StOCK_ID='{}';".format("GME"))[0][0]

10

In [8]:
import torch
from torch.utils.data import DataLoader
from datahelper import PostDataset, get_indices
from models import SentimentModel
import yaml

def get_config():
    with open("config.yml") as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
    return config['Inference']

indices = get_indices('GME', inference=True)

config = get_config()

post_data = PostDataset(config['embedding_max_len'], indices['post_ids'])

post_dataloader = DataLoader(dataset=post_data, 
                        batch_size=config['batch_size'], 
                        num_workers=config['num_workers'],
                        shuffle=config['shuffle']
                       )

connection established
[(datetime.datetime(2021, 5, 28, 19, 15, 26),)]
connection established
[(datetime.datetime(2021, 5, 28, 19, 15, 27),)]


In [9]:
sample = iter(post_dataloader).next()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [12]:
sample

{'post': ['Bought more GME cuz I’m retarded.',
  "I, too, YOLO'D my retirement account into GME when I switched jobs. PICK ME UP ON THE WAY TO THE MOO",
  '(O.C.) Keep shorting GME 🤡',
  'GME Great investment! Bought 600 more shares on my second account. 1600 shares in total! Life is gre',
  'Reviewing my GME investment'],
 'post_input_ids': tensor([[  101,  9326, 11266,  1167, 14748,  2036, 16408,  1584,   146,   787,
            182,  1231,  6817,  4902,   119,   102,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0, 

In [16]:
sample['post_attention_mask'].size()

torch.Size([5, 200])

In [ ]:
sample[]